<a href="https://colab.research.google.com/github/vvikasreddy/lexically_constrained_beam_search_/blob/main/beam_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

Marian MT model : https://huggingface.co/docs/transformers/model_doc/marian

Code to get the logits : https://huggingface.co/docs/transformers/main_classes/output

to get the BOS and EOS tokens: https://huggingface.co/docs/transformers/main_classes/configuration#transformers.PretrainedConfig.decoder_start_token_id

get topk values : https://pytorch.org/docs/stable/generated/torch.topk.html

ideas and core implementation drawn from this paper: https://arxiv.org/pdf/1704.07138

reference to link google colab with .py file from git : https://colab.research.google.com/github/jckantor/cbe61622/blob/master/docs/A.02-Downloading_Python_source_files_from_github.ipynb


## downloading essential modules

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Importing necessary libraries

In [2]:
import torch, random
from torch.utils.data import Dataset
from datasets import load_dataset
from transformers import MarianMTModel, MarianTokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torch.utils.data import DataLoader
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

## loading the dataset, considering the wmt turkish - english translation

In [3]:
ds = load_dataset("wmt/wmt16", "tr-en")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/11.1k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/36.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/153k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/466k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/205756 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3000 [00:00<?, ? examples/s]

## Glancing the organization of the dataset

In [4]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 205756
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 1001
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 3000
    })
})

In [5]:
ds['train'][0]

{'translation': {'en': "Kosovo's privatisation process is under scrutiny",
  'tr': "Kosova'nın özelleştirme süreci büyüteç altında"}}

## Loading the tokenizer and model, based of Marian-NMT

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tr-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tr-en")

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/840k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/797k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.56M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [7]:
model

MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(62389, 512, padding_idx=62388)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(62389, 512, padding_idx=62388)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [8]:
# taking a look at the number of beams used by the model.
model.config.num_beams

6

In [9]:
# viewing the turkish and english translation
print(ds["validation"][1]["translation"]["tr"])
print(ds["validation"][1]["translation"]["en"])

Norveç'in beş milyon insanı en yüksek yaşam standartlarının tadını çıkarıyor, sadece Avrupa'da değil, dünyada.
Norway's five million people enjoy one of the highest standards of living, not just in Europe, but in the world.


## Extracting and importing the constraints from my github

In [10]:
# code to import constraints and store in a local directory, from my git. (code reference cicted above.)

user = "vvikasreddy"
repo = "lexically_constrained_beam_search_"
pyfile = "constraints.py"

# i.e url is "https://github.com/vvikasreddy/lexically_constrained_beam_search_/blob/main/constraints.py"

url = f"https://raw.githubusercontent.com/{user}/{repo}/main/{pyfile}"
!wget --no-cache --backups=1 {url}

import constraints

--2024-12-06 16:59:56--  https://raw.githubusercontent.com/vvikasreddy/lexically_constrained_beam_search_/main/constraints.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4914 (4.8K) [text/plain]
Failed to rename constraints.py to constraints.py.1: (2) No such file or directory
Saving to: ‘constraints.py’

constraints.py      100%[===================>]   4.80K  --.-KB/s    in 0s      

2024-12-06 16:59:56 (48.9 MB/s) - ‘constraints.py’ saved [4914/4914]



In [11]:
# takes almost 4 minutes to get the constraints, you will see 3 progress bars
c = constraints.get_constraints()

100%|██████████| 26221852/26221852 [01:00<00:00, 434048.95it/s]


In [12]:
print("some of the constraints are :")

# Extract 5 random keys
random_keys = random.sample(list(c.keys()), 5)

for key in random_keys:
  print(key, c[key])

print("The length of the constraints is", len(c))

some of the constraints are :
('ICTY', 'ile') (('the', 'ICTY'), 0.9025797813529675)
("Stojanovska'nın", 'haberi') (('Stojanovska', 'for'), 0.9700478004217697)
('doğal', 'gaz') (('natural', 'gas'), 0.9617651934139755)
('küresel', 'ekonomik') (('global', 'economic'), 0.9527704811110207)
('İgor', "Jovanoviç'in") (('Igor', 'Jovanovic'), 1.0395630705897185)
The length of the constraints is 570


## Helper functions for Beam Search

In [13]:
def get_ngrams(src, n = 2, ):
  """
  Args:
    src: text for which ngrams should be returned
    n : represents the value of ngrams

  returns:
    returns ngrams
  """

  src = src.split(" ")
  src = [tuple(src[i:i+n]) for i in range(len(src) - n + 1)]
  return src

def constraints_tokens(src, c):
  """
  args:
    src: It is the turkish sentence, to which we want to return the constraints
    c: represents the entire list of constraints

  returns:
    returns the corresponding constraints of the src text
  """

  # gets the ngrams
  ngrams = get_ngrams(src)

  constraints_src = []
  for ngram in ngrams:

    # if ngram is present then add it to the constraints list
    if ngram in c:
      f = c[ngram][0]
      for gram in f:
        if  gram in constraints_src: continue
        out = tokenizer(gram, return_tensors="pt")
        constraints_src.append(out["input_ids"])

  return constraints_src

# this are some of the example constraints of the sample text in turkish
constraints_tokens("Southeast European Times için Priştine'den Muhamet Brayşori'nin haberi -- 21/03/12", c)

[tensor([[3113,   56,   47, 1517,    0]]),
 tensor([[5827, 1786,  373,    0]]),
 tensor([[5827, 1786,  373,    0]]),
 tensor([[3762,    0]]),
 tensor([[27,  0]]),
 tensor([[3113,   56,   47, 1517,    0]]),
 tensor([[21,  0]]),
 tensor([[ 4388, 10158,   204,     0]]),
 tensor([[1417,    0]]),
 tensor([[6041,   47, 2628,    0]]),
 tensor([[27,  0]]),
 tensor([[3113,   56,   47, 1517,    0]])]

In [14]:
def visualize_data(decoder_input):
  """
  code used for sanity, to visualize the constraints.
  args:
    decoder_input : takes in decoder token ids
  returns:
    returns the tokens"""

  return tokenizer.decode(decoder_input.squeeze(), skip_special_tokens = True)

In [16]:


def get_indices(ds, c):
  """
  Since wmt dataset is not domain speicific, I am trying to only take the sentences, for which constraints are present.

  args:
    ds: represents the hugging face WMT dataset
    c: represents the constraints
  returns:
    indices: which contains the first 1000 indices, that have constraints present.
  """
  count = 0
  indices = []
  count = 1000
  i = 0

  while count:
    x = ds["train"][i]["translation"]["tr"]
    if constraints_tokens(x, c):
      count -=1
      indices.append(i)
    i+=1

  return indices

print("The last index in all of the 3000 is,", get_indices(ds,c)[-1])

The last index in all of the 3000 is, 2102


In [24]:
class Results(Dataset):

  """
  Results class is the implementation of the Beam search

  Methods:

  generate_translation: Generates the next best token
  get_the_text: returns the translated text at the end of beam search
  get_top_k_prob: given a bunch of translations, it returns the top k beams
  constraints_tokens: returns the constraints of the particular target sentence (i.e in our case turkish sentence)
  beam_search: tries to return the best translation, we have a cap of maxlength 30, and beamsize of 6
  __len__: returns the length of indices considered (in our case 1000, same as the size of validation set)
  __getitem__: for each item called returns the translated sentence at the end of beam search
  """

  def __init__(self, ds, c, indices, device = None):
      """
      Args:
          ds : represents the Hugging face dataset
          c: represents all the constraints generated
          indices: these are the indices which we consider for the test
          device: chooses the device based on the configs
      """
      self.data = ds
      self.c = c
      self.indices = indices
      if device is None:
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  def generate_translation(self, src_text, decoder_input=[], probabilities=[], get_constrained_token_probability=-1, k=6):

    """
    this method generates the k best translations and their probabilites

    args:
      src_text : target sentence in turkish
      decoder_inputs : holds the token ids in english, up till that timestep
      probabliities : holds the corresponding probabilities of the decoder_inputs
      get_constrained_token_probablity: holds value -1, if there is no constraint, else contains the constraint token id
      k : represents the beam size, which is set to 6

    returns:
      decoder_input_ids: it holds the decoder input ids so far
      probabilities: holds the corresponding probabilities of the decoder inputs
      vis_data: used for sanity purposes, to visualize the generated data
      """

    # holds cuda if available else cpu
    device = self.device

    model.to(device)

    # Tokenize input
    encoder_inputs = tokenizer(src_text, return_tensors="pt").to(device)

    # If decoder_input is empty, include the decoder start token
    if len(decoder_input) == 0:
      # Initial decoder start token has probability 1
      probabilities = torch.tensor([[1.0]]).to(device)
      decoder_input = torch.tensor([[model.config.decoder_start_token_id]]).to(device)
    else:
      # Ensure decoder_input and probabilities are on the correct device
      decoder_input = decoder_input.to(device)
      if probabilities != [] :
        probabilities = probabilities.to(device)

    # Change the model to eval mode and stop the computation of gradients
    model.eval()
    with torch.no_grad():
      # Generate tokens
      outputs = model(
          input_ids=encoder_inputs.input_ids,
          attention_mask=encoder_inputs.attention_mask,
          decoder_input_ids=decoder_input
      )

      # Get the most frequently generated token
      next_token_logits = outputs.logits[:, -1, :]

      # Constraint handling
      if get_constrained_token_probability != -1:
          softmax_ = torch.softmax(next_token_logits, dim=-1)
          return softmax_[0][get_constrained_token_probability]

      # Get the top k tokens with maximum logits value
      top_probs, top_indices = torch.topk(torch.softmax(next_token_logits, dim=-1), k=k)

    # Initializing output containers
    decoder_input_tokens = []
    probs = []
    vis_data = []

    for indx, id in enumerate(top_indices[0]):
        # Concatenate new tokens and probabilities
        new_decoder_input = torch.cat([decoder_input, id.unsqueeze(0).unsqueeze(0)], dim=1)
        new_probs = torch.cat([probabilities, top_probs[0][indx].unsqueeze(0).unsqueeze(0)], dim=1)
        decoder_input_tokens.append(new_decoder_input)
        probs.append(new_probs)

        # Generate visualization data, used for sanity (removed for submission)
        vis_data.append((vis_data, tokenizer.decode(new_decoder_input.squeeze(), skip_special_tokens=True)))

    return decoder_input_tokens, probs, vis_data

  def get_the_text(self, x):
    """
    Given decoder token ids, it converts to tokens

    args:
      x: represents the decoder token ids
    returns:
      the corresponding translated sentences in english
    """

    token_id_list = x

    # moves tensors to cpu
    token_ids_batch = [tensor_item.cpu().squeeze().tolist() for tensor_item in token_id_list]

    decoded_sentences = tokenizer.batch_decode(token_ids_batch, skip_special_tokens=True)

    return decoded_sentences

  def get_top_k_prob(self, A, B, k=6):

    """
    takes in decoder inputs, and corresponding probablities and returns the k best beams

    args:
      A: represents the list of decoder input ids
      B: represents the list of correspondding probabilities
      k : it is the beam size
    returns:
      returns the k best values."""

    # container to hold the cummulative probablility sum of the beams
    d = {}

    # cummulative sum
    for indx, val in enumerate(B):
      cum_sum = torch.prod(val)
      d[cum_sum] = indx

    # sort to the cummulative probability sum
    sorted_keys = sorted(d.keys(), reverse = True)

    # containers to store top k values
    top_k_indices = []
    top_k_sequences = []

    for key in sorted_keys[:k]:
      top_k_indices.append(A[d[key]])
      top_k_sequences.append(B[d[key]])

    return top_k_sequences, top_k_indices

  def constraints_tokens(self, src):
    """
    args:
      src: It is the turkish sentence, to which we want to return the constraints
      c: represents the entire list of constraints

    returns:
      returns the corresponding constraints of the src text
  """
    # gets the ngrams
    ngrams = get_ngrams(src)
    constraints_src = []

    for ngram in ngrams:
      # if ngram is present then add it to the constraints list
      if ngram in self.c:
        f = self.c[ngram][0]
        for gram in f:

          if  gram in constraints_src: continue
          out = tokenizer(gram, return_tensors="pt")
          constraints_src.append(out["input_ids"])

    return constraints_src

  def beam_search(self, maxlen, numC, k, src, constrained_tokens):

    """
    Combines all the methods to return the k best beams

    args:
      maxlen: it is the max length, until where we generate sentences
      numC: represents the number of constraints
      k: represents the number of beams
      src: represents the target sentences, in turkish language
      constrained_tokens: represnts the constraints associated with the corresponding sentence

    returns:
      returns 2 sentences,
      1. sentence containing the constraints associated with the sentence
      2. sentence generated without any constraints
    """

    # device where values are present
    device = self.device

    decoder_start_token = model.config.decoder_start_token_id

    # initialize the grids
    grids = [[[] for _ in range(numC + 1)] for _ in range(maxlen + 1)]
    probs_grid  = [[[] for _ in range(numC + 1)] for _ in range(maxlen + 1)]

    # intialize the first grid to start hyp
    grids[0][0] = [1]

    generated_constraint_index = 0

    # iterate through the timestep
    for t in range(1, maxlen):

        index_c = max(0, (numC - t) - maxlen)
        # iterate through constraints
        for c in range(index_c, min(t, numC) + 1):

            # Prepare batched generation to reduce individual calls, g holds the decoder input tokens, for the current timestep.
            g = []

            # storing decoder inputs
            decoder_inputs = []
            probs = []
            vis_data = []

            # generation of translations for current hypotheses
            for indx, element in enumerate(grids[t-1][c]):

              if type(element) == int:
                decoder_input = []
                prev_probs =[]
              else:
                decoder_input = element.to(device)
                prev_probs = probs_grid[t-1][c][indx].to(device)

              # collecting the batch of translations
              t_g, t_probs, t_vis_data = self.generate_translation(src_text=src, decoder_input=decoder_input, probabilities=prev_probs)

              # adding to the current lists
              g.extend(t_g)
              probs.extend(t_probs)
              vis_data.extend(t_vis_data)

            # retrieve the probability of the constraint and add that to the decoder_input.
            if c > 0 and constrained_tokens:

              for indx, element in enumerate(grids[t-1][c-1]):

                if c == 1 and t == 1:
                  decoder_inputs = torch.tensor([[model.config.decoder_start_token_id]]).to(device)
                  prob = torch.tensor([[1]]).to(device)
                else:
                  decoder_inputs = element.to(device)
                  prob = probs_grid[t-1][c-1][indx].to(device)

                # iterating, because a constraint can be made up of many token ids
                partial_constraints = constrained_tokens[c - 1].tolist()

                # iterating, because a constraint can be made up of many token ids
                for partial_constraint in partial_constraints[0]:
                  if partial_constraint == 0: continue


                  cons = self.generate_translation(src, decoder_input=decoder_input, get_constrained_token_probability=partial_constraint)
                  decoder_inputs = torch.cat([decoder_inputs, torch.tensor(partial_constraint).unsqueeze(0).unsqueeze(0).to(device)], dim=1)
                  prob = torch.cat([prob, torch.tensor(cons).unsqueeze(0).unsqueeze(0).to(device)], dim=1)

                  # appending to generated token ids (g), and probabilites to prob (probs)
                  g.append(decoder_inputs)
                  probs.append(prob)

            # storing the k best token ids to the current grid.
            probs_grid[t][c], grids[t][c] = self.get_top_k_prob(g, probs, k)

    # return the best token ids which contains all the constraints, and ones without any constraints.
    return self.get_top_k_prob(grids[maxlen -1][numC], probs_grid[maxlen - 1][numC], k = 1), self.get_top_k_prob(grids[maxlen -1][0], probs_grid[maxlen - 1][0], k = 1)

  def __len__(self):
      """Returns the total number of samples in the dataset."""
      return len(self.indices)

  def __getitem__(self, idx):
      """
      Retrieves a single target sentence, and returns prediction with constraints, prediction without any constraints, and reference translation

      args:
          idx: Index of the sample to retrieve.

      Returns:
          returns target sentence, prediction with constraints, prediction without any constraints, and reference translation
      """

      sample = self.data["test"][self.indices[idx]]["translation"]["tr"]

      # gets the constraints for that sentence
      constraints = self.constraints_tokens(sample)

      # using beam search gets the predicted translations, both with and without constraints
      prediction, prediction_without_constraints = self.beam_search(maxlen= 30, numC=len(constraints), k = 4, src = sample, constrained_tokens = constraints)

      # convert the token ids to tokens
      prediction = self.get_the_text(prediction[1])
      prediction_without_constraints = self.get_the_text(prediction_without_constraints[1])
      result = self.data["test"][self.indices[idx]]["translation"]["en"]

      return sample, prediction[0], prediction_without_constraints[0], result

In [25]:
# creating the instance of the Results class
results = Results(ds,c, indices = get_indices(ds,c))

# generating sentences for 100th index
results[100]

1000

In [ ]:
# Initialize lists to store predictions and actual texts
predictions = []
predictions_without_constraints = []
references = []

# using a dataloader, to get the batches of 10
dataloader = DataLoader(results, batch_size=10, shuffle=False)

# Iterate through dataloader and storing results in containers
for tr_text, pred_text, pred_cons_text, actual_text in tqdm(dataloader):

  predictions.extend([ref.split() for ref in pred_text])
  predictions_without_constraints.extend([ref.split() for ref in pred_cons_text])
  references.extend([[ref.split()] for ref in actual_text])



In [ ]:
from nltk.translate.bleu_score import sentence_bleu
import nltk

def calculate_bleu_score(references, translations):
    """
    Calculate BLEU score for multiple references and translations.

    Args:
    - references: List of reference translations
      (each reference is a list of lists of tokens)
    - translations: List of machine translations
      (each translation is a list of tokens)

    Returns:
    - Average BLEU score
    """
    bleu_scores = []

    for ref, trans in zip(references, translations):
        # Calculate BLEU score for this sentence
        score = sentence_bleu(ref, trans)
        bleu_scores.append(score)


    # Calculate and return average BLEU score
    return sum(bleu_scores) / len(bleu_scores)


bleu_score_constraints = calculate_bleu_score(references, predictions)
bleu_score_without_constraints = calculate_bleu_score(references, predictions_without_constraints)
print(f"BLEU Score with constraints: {bleu_score_constraints:.4f}, BLEU score without constraints: {bleu_score_without_constraints:.4f}")

In [ ]:
import locale
print(locale.getpreferredencoding())
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

reference to get this working : https://stackoverflow.com/questions/56081324/why-are-google-colab-shell-commands-not-working

In [ ]:
!pip install rouge-score


In [ ]:
from rouge_score import rouge_scorer

def calculate_rouge_score(references, translations):
    """
    Calculate average ROUGE-1, ROUGE-2, and ROUGE-L F-scores for multiple references and translations.

    Args:
    - references: List of reference translations
      (each reference is a list of lists of tokens; we'll use the first reference set if multiple)
    - translations: List of machine translations
      (each translation is a list of tokens)

    Returns:
    - A dictionary with average ROUGE-1, ROUGE-2, and ROUGE-L F-scores.
    """
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge1_scores = []
    rouge2_scores = []
    rougeL_scores = []

    for ref_list, trans in zip(references, translations):
        # If you have multiple references per sentence, we take the first one
        # ref_list is something like: [["this", "is", "a", "reference"], ...]
        ref_str = " ".join(ref_list[0])  # first reference
        trans_str = " ".join(trans)

        scores = scorer.score(ref_str, trans_str)
        rouge1_scores.append(scores['rouge1'].fmeasure)
        rouge2_scores.append(scores['rouge2'].fmeasure)
        rougeL_scores.append(scores['rougeL'].fmeasure)

    return {
        'rouge1': sum(rouge1_scores) / len(rouge1_scores),
        'rouge2': sum(rouge2_scores) / len(rouge2_scores),
        'rougeL': sum(rougeL_scores) / len(rougeL_scores)
    }

# Example usage (assuming `references` and `predictions` and `predictions_without_constraints` are defined):
rouge_scores_constraints = calculate_rouge_score(references, predictions)
rouge_scores_no_constraints = calculate_rouge_score(references, predictions_without_constraints)

print(f"ROUGE-1 with constraints: {rouge_scores_constraints['rouge1']:.4f}, ROUGE-1 without constraints: {rouge_scores_no_constraints['rouge1']:.4f}")
print(f"ROUGE-2 with constraints: {rouge_scores_constraints['rouge2']:.4f}, ROUGE-2 without constraints: {rouge_scores_no_constraints['rouge2']:.4f}")
print(f"ROUGE-L with constraints: {rouge_scores_constraints['rougeL']:.4f}, ROUGE-L without constraints: {rouge_scores_no_constraints['rougeL']:.4f}")
